In [1]:
import numpy as np
import pprint
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [2]:
pp = pprint.PrettyPrinter(indent=2)
env = GridworldEnv()

In [3]:
def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    V = np.zeros(env.nS)
    V_prev = np.zeros(env.nS)
    deltas = []
    converged = False
    while not converged:
        
        delta = 0
        for s in range(env.nS):
            value = 0
            #v_current = 
            # iterate over possible actions
            for a, pr_a in enumerate(policy[s]):
                # iterate over possible successor states
                for pr_transition, s_next, r, done in env.P[s][a]: 
                    value += pr_a * pr_transition * (r + discount_factor * V[s_next])
            delta = max(delta, V[s] - value)
            V[s] = value
            
        deltas.append(delta)
        if delta < theta:
            converged = True
        
    return np.array(V)

Use Bellman optimality equation for policy improvement: $v_{\ast}(s) = \max_{a \in A(s)} q_{\pi}(s,a) = \max_{a \in A(s)} \sum_{s',r} p(s',r|s,a)[r + \gamma v_{\ast}(s')$; action chosen is simply the argmax

In [8]:
def policy_improvement(env, policy_eval_fn=policy_eval, discount_factor=1.0):
    """
    Policy Improvement Algorithm. Iteratively evaluates and improves a policy
    until an optimal policy is found.
    
    Args:
        env: The OpenAI envrionment.
        policy_eval_fn: Policy Evaluation function that takes 3 arguments:
            policy, env, discount_factor.
        discount_factor: gamma discount factor.
        
    Returns:
        A tuple (policy, V). 
        policy is the optimal policy, a matrix of shape [S, A] where each state s
        contains a valid probability distribution over actions.
        V is the value function for the optimal policy.
        
    """
    # compute q(s,a) for each action a in a given state s
    def get_qs(s, V):
        """
        Args: 
            s: current state
            V: current value function
        
        Returns:
            qs: vector of current action-value estimates for each possible action 
        """
        qs = np.zeros(env.nA)
        
        # iterate over possible actions
        for a in range(env.nA):
            # for each outcome of action in current state
            for trans_prob, next_state, reward, done in env.P[s][a]:
                qs[a] += trans_prob * (reward + discount_factor * V[next_state])
        
        return qs
    
    
    # Start with a random policy
    policy = np.ones([env.nS, env.nA]) / env.nA
    
    
    while True:
        V = policy_eval(policy, env, discount_factor)
        policy_stable = True
        # iterate over each state
        for s in range(env.nS):
            # action given current policy
            current_a = np.argmax(policy[s])
            
            # get q(s,a) for all a
            qs = get_qs(s, V)
            
            # set new deterministic policy 
            policy[s] = np.zeros(env.nA)
            new_a = np.argmax(qs)
            policy[s, new_a] = 1
            
            if new_a != current_a:
                policy_stable = False
    
        if policy_stable:
            return policy, V

In [9]:
policy, v = policy_improvement(env)
print("Policy Probability Distribution:")
print(policy)
print("")

print("Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):")
print(np.reshape(np.argmax(policy, axis=1), env.shape))
print("")

print("Value Function:")
print(v)
print("")

print("Reshaped Grid Value Function:")
print(v.reshape(env.shape))
print("")



Policy Probability Distribution:
[[ 1.  0.  0.  0.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  1.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 1.  0.  0.  0.]]

Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):
[[0 3 3 2]
 [0 0 0 2]
 [0 0 1 2]
 [0 1 1 0]]

Value Function:
[ 0. -1. -2. -3. -1. -2. -3. -2. -2. -3. -2. -1. -3. -2. -1.  0.]

Reshaped Grid Value Function:
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]



In [10]:
# Test the value function
expected_v = np.array([ 0, -1, -2, -3, -1, -2, -3, -2, -2, -3, -2, -1, -3, -2, -1,  0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)